In [1]:
# import modules

import numpy as np
import pandas as pd
from pylab import *
from scipy.optimize import minimize, fmin_l_bfgs_b

In [2]:
# Load data

df = pd.read_csv('./data/ml-latest-small/ratings.csv')
df = df[(1 <= df.userId) & (df.userId <= 5)]
df.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [3]:
# Convert to a rating matrix

ratings = df.pivot_table(index='userId', columns='movieId', values='rating', fill_value=0)
Y = ratings.to_numpy()
Y.shape

(5, 481)

In [4]:
# Make User-Item matrix

User = Y.copy()
User -= User.mean()
User.shape

(5, 481)

In [5]:
# Make Item-User matrix

Item = User.copy().T
Item.shape

(481, 5)

In [ ]:
# Bayesian GPVLM model

import GPy
import matplotlib.pyplot as plt

input_dim = 2

kernel = GPy.kern.RBF(input_dim, ARD=True) + GPy.kern.White(input_dim)
model = GPy.models.BayesianGPLVM(Item, input_dim, kernel=kernel, num_inducing=30)
model.optimize(messages=True, max_iters=1e3)

In [ ]:
# Plot latent variables

model.plot_latent()

In [ ]:
# Sampling

user_id = 4

X_mean = model.X.mean
X_var = model.X.variance

X_mean_i = X_mean[Y[user_id] != 0, :]
X_var_i = X_var[Y[user_id] != 0, :]

x_min = min(X_mean_i[:, 0])
x_max = max(X_mean_i[:, 0])
y_min = min(X_mean_i[:, 1])
y_max = max(X_mean_i[:, 1])

samples = []
for n, mu in enumerate(X_mean_i):
    cov = np.diag(X_var_i[n])
    samples.append(np.random.multivariate_normal(mu, cov, 10000))

x, y = np.vstack(samples).T

In [ ]:
# Plot the latent variables from a user

import numpy as np
import matplotlib.pyplot as plt

fig, (axL, axR) = plt.subplots(ncols=2, figsize=(12,5), sharex=True)

axL.scatter(X_mean_i[:, 0], X_mean_i[:, 1])

H = axR.hist2d(x,y, bins=20, cmap='jet') #, range=[[x_min, x_max], [y_min, y_max]])
axR.set_xlabel('x')
axR.set_ylabel('y')
#fig.colorbar(H[3],ax=axR)

plt.show()